<a href="https://colab.research.google.com/github/Herrera1022/Proyecto-Kaggle/blob/main/04_modelo_con_preprocesado_de_otra_forma_y_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ===============================================================
# 04 - modelo con preprocesado de otra forma y Random Forest.ipynb
# ===============================================================

# En este notebook entrenamos un modelo Random Forest con otro tipo de
# preprocesado (sin escalar, imputación diferente)
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# ================================
# 1. Cargar datos
# ================================
train = pd.read_csv("/kaggle/input/dataset/train.csv")
test = pd.read_csv("/kaggle/input/dataset/test.csv")

# Remover ID
X = train.drop(["RENDIMIENTO_GLOBAL", "ID"], axis=1)
y = train["RENDIMIENTO_GLOBAL"]

test_ids = test["ID"]
test_features = test.drop(["ID"], axis=1)

# ================================
# 2. Preprocesamiento
# ================================
numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(include="object").columns

numerical_transformer = SimpleImputer(strategy="median")

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinal", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ],
    remainder="drop"
)

# Transformar
X_processed = preprocessor.fit_transform(X)
test_processed = preprocessor.transform(test_features)

# ================================
# 3. División train/valid
# ================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

# ================================
# 4. Random Forest rápido
# ================================
rf = RandomForestClassifier(
    n_estimators=80,     # antes 300, demasiado pesado
    max_depth=20,        # limita el tamaño del RF
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
print("Accuracy Random Forest:", acc)

# ================================
# 5. Generar submission
# ================================
test_preds = rf.predict(test_processed)

submission = pd.DataFrame({
    "id": test_ids,
    "label": test_preds
})

submission.to_csv("submission_rf.csv", index=False)
print("Archivo submission_rf.csv generado.")


Columns in train DataFrame: Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA F_ESTRATOVIVIENDA F_TIENEINTERNET F_EDUCACIONPADRE F_TIENELAVADORA F_TIENEAUTOMOVIL E_PRIVADO_LIBERTAD E_PAGOMATRICULAPROPIO F_TIENECOMPUTADOR F_TIENEINTERNET.1 F_EDUCACIONMADRE RENDIMIENTO_GLOBAL INDICADOR_1 INDICADOR_2 'INDICADOR_3', 'INDICADOR_4'],dtype='object')
Accuracy SVM: 0.7061732851985559Archivo submission_svm.csv generado correctamente.
